In [308]:
import requests
from bs4 import BeautifulSoup

import random
import re
import pandas as pd
import numpy as np

from datetime import datetime


In [309]:
wiki_link = 'https://en.wikipedia.org/wiki/2025_in_hip-hop'

headers = {
            'User-Agent': random.choice([
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
                'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0)',
            ])
        }

html_text = requests.get(wiki_link, headers=headers).text
soup = BeautifulSoup(html_text, 'lxml')

In [310]:
wiki_scrape = soup.select('div.mw-heading.mw-heading3,table.wikitable')
#now we want to associate each table with a date. could do {Date:Table}

In [311]:
wiki_scrape[3].find('h3').get('id')

'January_2'

In [312]:
data = {}
prev_header = None

for n in wiki_scrape:
    if n.get('class') == ['mw-heading', 'mw-heading3']:
        prev_header = n.find('h3').get('id')
    elif n.get('class') == ['wikitable']:
        data[prev_header] = pd.read_html(str(n))[0] 

how do we want the data? I guess it would be nice to be one table. Month, Day, Artist, Album, Record Label.
Then we can merge on Month?

In [313]:
df_lst = []
for month, table in data.items():
    table['Month'] = re.sub('[^a-zA-Z]+', '', month)
    df_lst.append(table)

In [314]:
albums_release_data = pd.concat(df_lst)#.to_csv('out.csv', index = False)

In [315]:
albums_release_data = albums_release_data.drop(columns=['Ref.', 'Entering chart position'])

In [316]:
albums_release_data['Month'] = albums_release_data['Month'].replace('UnscheduledandTBA', np.nan)

In [ ]:
albums_release_data['Day'] = albums_release_data['Day'].replace('TBA', np.nan)

In [320]:
albums_release_data = albums_release_data[['Artist(s)','Album','Record label(s)', 'Month', 'Day']]

In [ ]:
print(albums_release_data.to_string())
#how should multiple artists/record labels be handled? list?
#create new table for any list when data modeling

                                     Artist(s)                                                         Album                                                         Record label(s)     Month   Day
0                                    03 Greedo                                2025: The Streetz is Over Wiit                                                       Self-released[13]   January     1
1                                  Duwap Kaine                                  Nobody Is Coming To Save You                                                                 TAF[14]   January     1
2                      Boldy James & RichGains                                    Murder During Drug Traffic                                                       Self-released[15]   January     3
3                                   Jon Connor                                             Food for the Soul                                                       Self-released[16]   January     3
4              